# day11: Backpropagation

# Objectives

* Walk through backpropagation demo
* Manually try backpropagation for a sample input
* Compare your result to the implementation of backprop in sklearn

# Outline

* [Part 1: Backprop demo from Google's ML crash course](#part1)
* [Part 2: Manual do forward and backward pass on simple input](#part2)
* [Part 3: Compare Part 2 results with sklearn implementation](#part3)

We expect you'll get through at least first 2 parts during the in-class session.


# Takeaways

* Backpropagation is a way to compute the gradients of composable functions
* Running backpropagation requires:
* --- Allocating storage at each node in computation graph
* --- Running forward pass to compute inputs and outputs of each node
* --- Running backward pass to compute gradients wrt each node's inputs and outputs, using the chain rule
* --- Computing each parameter's gradients using results of backward pass
* Sklearn uses backpropagation within its `MLPRegressor` and `MLPClassifier` implementations to compute the gradients it uses in its gradient descent algorithms

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sklearn.neural_network

In [3]:
# import plotting libraries
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Part 1: Read through the backprop demo from Google's ML crash course

This demo considers a simple neural net that is like a general MLP, with

* hidden_layer_sizes = [2, 2]
* No bias parameters (they are fixed to zero and never updated)
* general purpose activation function

Open this link in your browser, and scroll through each step

<https://developers-dot-devsite-v2-prod.appspot.com/machine-learning/crash-course/backprop-scroll>

### Discussion 1a: Which update steps of the backward pass use the chain rule?

### Discussion 1b: What would you need to change to do backprop for binary classification with log loss instead of regression with squared error loss

# Part 2: Try out manually computing the gradient using backprop

Use the following *simpler* network architecture, which is like the demo above but only uses one hidden layer (not two)

* Terminal node loss function: $\frac{1}{2}(\hat{y}_{\text{output}} - y_{\text{target}})^2$
* Hidden layer sizes: [2]
* Hidden activation: ReLU


You might need these elementary derivatives:

$$
\frac{d}{d y} \frac{1}{2}( y - m)^2  = y - m
$$

$$\frac{d}{d x} \text{ReLU}(x) =
\begin{cases}
    1 \quad \text{if} ~ x > 0 \\
    0 \quad \text{otherwise}
\end{cases}
$$

Consider running the algorithm with the following inputs:
    
### Data:
    
* $x_{\text{in}} = 1.0$
* $y_{\text{target}} = 1.0$

### Parameters for Layer 1 (input-to-hidden):

First neuron:

* $w_{11} = 1.0$
* $b_{11} = 0.0$

Second neuron:

* $w_{12} = 2.0$
* $b_{12} = 0.0$

### Parameters for Layer 2 (hidden-to-output):

* $w_{21} = 1.0$
* $w_{22} = 1.0$
* $b_2 = 0.0$


### Exercise 2a: Forward pass

What is the *output* of each node? Work it out manually and write your answer below.


TODO:

* Layer 1 Neuron 1 output: ____
* Layer 1 Neuron 2 output: ____
* Layer 2 Output: ____

* Terminal node output:  ____

### Exercise 2b: Backward pass for layer 2

What is gradient of E wrt the Layer 2 output?

What is gradient of E wrt the Layer 2 input?


### Exercise 2c: Backward pass for layer 1

What is gradient of E wrt each neuron's output in Layer 1?

What is gradient of E wrt each neuron's input in Layer 1?

### Exercise 2d: What is gradient wrt each weight parameter in Layer 2?

You can skip the bias parameters.

### Exercise 2e: What is gradient wrt each weight parameter in Layer 1?

You can skip the bias parameters.

# Part 3: Check your Part 2 answer with the result of sklearn

We can use sklearn to compute every result you did in Part 2.

Use this section to check your answer, and get a quick understanding of how sklearn's MLP handles forward and backward passes.

### Create an example input-output data pair

* x is a scalar, but we reshape to (1,1) to meet sklearn's 2-dim requirement for input arrays
* y is a scalar, but we reshpae to (1,) to meet sklearn's 1-dim requirement for output arrays

In [4]:
x_11 = np.asarray([1.0]).reshape(1,1)
y_1 = np.asarray([1.0])

### Create the MLP object and initialize it for the give problem

We set max_iter = 1 so that fit really just initializes all arrays to the correct size, 
and then quits.

We fully expect a "Convergence Warning". You can just ignore this.

In [5]:
mlp = sklearn.neural_network.MLPRegressor(
    hidden_layer_sizes=[2],
    activation='relu',
    alpha=0.0, max_iter=1)

In [6]:
mlp.fit(x_11, y_1);

/Users/mhughes/miniconda3/envs/comp135_2020f_env/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### Now, let's fill in our weights with desired values

In [7]:
# First layer

mlp.coefs_[0][:] = np.asarray([[1.0, 2.0]])

mlp.intercepts_[0][:] = np.asarray([0.0, 0.0])

In [8]:
# Second layer 

mlp.coefs_[1][:] = np.asarray([[1.0], [1.0]])

mlp.intercepts_[1][:] = np.asarray([0.0])

### Allocate storage for forward pass

In [9]:
# Initialize storage for forward pass
outputs_per_node = [x_11] + [None] * (mlp.n_layers_ - 1)

# Initialize storage for the backward pass
deltas_per_node = [None] * (len(outputs_per_node) - 1)
coef_grads = [np.zeros_like(w_arr) for w_arr in mlp.coefs_]
intercept_grads = [np.zeros_like(b_arr) for b_arr in mlp.intercepts_]

### Do the forward pass

Uses internal sklearn code to evaluate output of each node in our computation graph

In [10]:
# Remember, our first "output" is just the input x_11, so that's being fed in implicitly
# Unusually, sklearn does not compute the terminal node output here, that's done below
outputs_per_node = mlp._forward_pass(outputs_per_node)

### Do the backward pass

Uses internal sklearn code to evaluate the gradients at each coef value and each intercept value

In [11]:
terminal_node_loss, coef_grads, intercept_grads = mlp._backprop(
    x_11, y_1,
    outputs_per_node, deltas_per_node,
    coef_grads, intercept_grads)

### Exercise 3a: Show results of forward pass

In [12]:
print("Output of layer 1:")
print(outputs_per_node[1])

print("Output of layer 2:")
print(outputs_per_node[2])

print("Output of terminal node:")
print(terminal_node_loss)

Output of layer 1:
[[1. 2.]]
Output of layer 2:
[[3.]]
Output of terminal node:
2.0


### Result 3b: Show results of grad wrt inputs of Layer 2

In [13]:
print("What is gradient of E wrt each neuron's input value in Layer 2?")
print(deltas_per_node[1])

What is gradient of E wrt each neuron's input value in Layer 2?
[[2.]]


### Result 3c: Show results of grad wrt inputs of Layer 1

In [14]:
print("What is gradient of E wrt each neuron's input value in Layer 1?")
print(deltas_per_node[0])


What is gradient of E wrt each neuron's input value in Layer 1?
[[2. 2.]]


### Result 3d: Show results of grad wrt WEIGHTS for Layer 2

In [15]:
print("Grad wrt weight coefs for layer 2")
print(coef_grads[1])

Grad wrt weight coefs for layer 2
[[2.]
 [4.]]


### Result 3e: Show results of grad wrt WEIGHTS for Layer 1

In [16]:
print("Grad wrt weight coefs for layer 1")
print(coef_grads[0])


Grad wrt weight coefs for layer 1
[[2. 2.]]


### Bonus Result: Show results of grad wrt BIAS PARAMETERS for Layer 2

In [17]:
for layer_id in range(mlp.n_layers_ - 1):
    print("Grad wrt intercept coefs layers %d" % (layer_id+1))
    print(intercept_grads[layer_id])
    print()

Grad wrt intercept coefs layers 1
[2. 2.]

Grad wrt intercept coefs layers 2
[2.]

